In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
# Load the data into a Pandas DataFrame
df = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [4]:
# Generate summary statistics
df.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [5]:
# Plot your data to see what's in your DataFrame
df.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [6]:
# Scale the numeric columns
scaled_numeric = StandardScaler().fit_transform(df)

# Convert scaled data to a DataFrame
scaled_df = pd.DataFrame(scaled_numeric, columns=df.columns)
scaled_df.set_index(df.index, inplace=True)
scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [48]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))
inertia = []

# Fill list of inertias for range of k values
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(scaled_df)
    inertia.append(k_model.inertia_)
    
# Plot Elbow Curve
df_elbow = pd.DataFrame({"k": k, "inertia": inertia})
kmeans_elbow_plot = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)
kmeans_elbow_plot

:Curve   [k]   (inertia)

# Analysis

4 appears to be the best value for k. It is the most significant bend in the graph and it is a nearly straight line afterwards

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [26]:
# Model the data using the KMeans clustering algorithm using prefered number of clusters

# Set number of clusters to 4 (based on elbow graph)
k = 4

# Model and fit the data
model = KMeans(n_clusters=k, random_state=1)
model.fit(scaled_df)

# Make and display predictions from the model
kmeans_predictions = model.predict(scaled_df)
kmeans_predictions

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [57]:
# Add predictions to copy of the original DataFrame
predict_df = df.copy()
predict_df['kmeans'] = kmeans_predictions
predict_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,kmeans
coin_id,,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761,2
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023,2
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954,0
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193,0
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384,2


In [58]:
# Create a scatter plot of predicted clusters
kmeans_clusters = predict_df.hvplot.scatter(
    x="price_change_percentage_24h", 
    y="price_change_percentage_7d", 
    by="kmeans",
    hover_cols='coin_id'
)
kmeans_clusters

:NdOverlay   [kmeans]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [33]:
# Create a PCA model with 3 components
pca = PCA(n_components=3)
pca_fit = pca.fit_transform(scaled_df)
pca_fit[0:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [59]:
# Display explained variance ratio for the PCA model
print(pca.explained_variance_ratio_, pca.explained_variance_ratio_.sum())

[0.3719856  0.34700813 0.17603793] 0.895031657030984


# Analysis
The Explained Variance Ratio values of " [0.3719856, 0.34700813, 0.17603793] " mean that:\
The first component explains ~37.2% of the data, \
The second component explains ~34.7% of the data \
And the last component explains ~17.6% of the data.

The sum of all three components explains ~89.5% of the data which should be sufficient for our purposes. 

In [36]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(
    pca_fit,
    columns=["PCA1", "PCA2", "PCA3"]
)
pca_df.set_index(df.index, inplace=True)
pca_df.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [47]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))
inertia = []

# Fill list of inertias for range of k values
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(pca_df)
    inertia.append(k_model.inertia_)
    
# Plot Elbow Curve
df_elbow = pd.DataFrame({"k": k, "inertia": inertia})
pca_elbow_plot = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)
pca_elbow_plot

:Curve   [k]   (inertia)

# Analysis

Based on the elbow graph, the best value for k seems to be 4. This is consistant with our findings from the first elbow graph made from the k-means predictions

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [60]:
# Model the data using the KMeans clustering algorithm using prefered number of clusters

# Set number of clusters to 4 (based on elbow graph)
k = 4

# Model and fit the data
model = KMeans(n_clusters=k, random_state=1)
model.fit(pca_df)

# Make and display predictions from the model
kmeans_predictions = model.predict(pca_df)
kmeans_predictions

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [42]:
# Make DataFrame of PCA components and KMeans predictions based on the PCA model
pca_predict_df = pca_df.copy()
pca_predict_df['kmeans'] = kmeans_predictions
pca_predict_df.head()

,PCA1,PCA2,PCA3,kmeans
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [52]:
# Create a scatter plot using hvPlot
pca_clusters = pca_predict_df.hvplot.scatter(
    x="PCA1", 
    y="PCA2", 
    by="kmeans",
    hover_cols='coin_id'
)
pca_clusters

:NdOverlay   [kmeans]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [50]:
# Composite plot to contrast the Elbow curves
kmeans_elbow_plot * pca_elbow_plot

:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [55]:
# Composite plot to contrast the clusters
kmeans_clusters + pca_clusters

:Layout
   .NdOverlay.I  :NdOverlay   [kmeans]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [kmeans]
      :Scatter   [PCA1]   (PCA2,coin_id)

# Analysis

Although both of the elbow graphs imply that 4 is the best value for k, it does not seem very meaningful to have ethlend and celsius-degree-token in their own respective groups. Having a k-value of 3 or even 2 seems more meaningful to the context of the data